In [2]:
import torch
from inter_vae_0110 import InteractionVAE
from Smiles_Vector_Dataset import Smiles_Vector_Dataset
from inter_vae_0110 import InteractionVAE

smiles_train_path = '../data/AKT1/AKT1_train.pt'
smiles_valid_path = '../data/AKT1/AKT1_valid.pt'
trained_inter_vae_path = './AKT1/'

train_data = torch.load(smiles_train_path)
val_data = torch.load(smiles_valid_path)
tensor_len = len(train_data[0][1])
print(f'tensor_len: {tensor_len}')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/nakamura/miniconda3/envs/iev2mol/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor_len: 207


In [2]:
int_vae = InteractionVAE(device, tensor_len).to(device)
ckpt = torch.load('./AKT1/inter_vae_200.pt')
int_vae.load_state_dict(ckpt)

<All keys matched successfully>

In [ ]:
int_vae.forward()

In [54]:
int_vae.eval()
zs = []
with torch.no_grad():
    val_loss = 0
    val_inter_recon_loss = 0
    val_inter_kl_loss = 0
    for val_batch_idx, (_, input_vecs) in enumerate(
        torch.utils.data.DataLoader(
            train_data,
            batch_size=128,
            shuffle=True,
            drop_last=True,)
        ):
        input_vecs = input_vecs.to(device)
        # ロス計算
        output_vecs, z, tmp_inter_kl_loss = int_vae.forward(input_vecs)
        zs.append(z)

In [55]:
z_list = []
for i in range(len(zs)):
    z_list.extend(z[i].tolist())

In [56]:
len(z_list)

1344

In [57]:
import numpy as np
np_zs = np.array(z_list)
mean = np.mean(np_zs, axis=0)
var = np.var(np_zs, axis=0)

In [58]:
print(f'mean: {mean}')
print(f'var: {var}')

mean: -0.04007825885006847
var: 0.9822239632257201


In [5]:
train_path = '../data/smitensor_iev_train_dataset_tensorized_by_smiles_vae_dmqp1m_no_dot_dup.pt'
train_dataset = torch.load(train_path)
len(train_dataset)


8340